In [2]:
%load_ext autoreload
%autoreload 2

%autosave 120
import sys
import pandas as pd
import glob
import numpy as np
import os
import random
from tqdm import tqdm
import json
from modules.utilities import are_bugs_from_tangled, has_bug, is_same_history, parse_hash_delimited_string
from modules.Untangler import Untangler

Autosaving every 120 seconds


In [ ]:
import time


untangler = Untangler(model_name="openai")
untangler.change_model("gpt-4o")

csv_files = glob.glob("./data/Processed/*.csv")
for csv_file in tqdm(csv_files):
    df = pd.read_csv(csv_file,delimiter="\t")
    project_name = os.path.basename(csv_file).replace(".csv","")

    rows = []
    for _, row in tqdm(df.iterrows()):
        row = row.to_dict()
        history = parse_hash_delimited_string(row["Buggycommiit"])
        change_list = parse_hash_delimited_string(row["TangledWMoveandFileRename"])
        if row["Age"] < (365*2):
            continue
        if sum(history) >= 1 and any(t > 1 for t in [a * b for a, b in zip(change_list, history)]):
            row["Decision"] = "Buggy"
            change_list.reverse()
            history.reverse()
            
            json_path = os.path.join("./data/source-methods/", project_name, row["file"])
            data = json.load(open(json_path, encoding="utf8"))

            buggy = False

            for index, (change_count, is_buggy) in enumerate(zip(change_list, history)):
                if is_buggy == 1 and change_count > 1:
                    hash = data["changeHistory"][index]
                    
                    commit_data = data["changeHistoryDetails"][hash]
                    change_data = commit_data["subchanges"][0] if "subchanges" in commit_data.keys() else commit_data

                    untangler.prepare_prompt(change_data["commitMessage"], change_data["diff"])

                    error = True
                    while error:
                        try:
                            result = untangler.detect()
                            error = False
                        except Exception as e:
                            error = True
                            print(f"Error - {e}.\nRetrying in 10s")
                            time.sleep(10)
                    if result == "Buggy":
                        buggy = True
                        break
                elif is_buggy == 1 and change_count == 1:
                    buggy = True
                    break
                    
            if buggy:
                row["Detection"] = "Buggy"
            else:
                row["Detection"] = "NotBuggy"
            row["result"] = result
        elif sum(history) >= 1:
            row["Decision"] = "Buggy"
            row["Detection"] = "Buggy"
        else:
            row["Decision"] = "NotBuggy"
            row["Detection"] = "NotBuggy"
        rows.append(row)
    df = pd.DataFrame(rows)
    df = df.drop(["Unnamed: 26"], axis=1)
    df.to_csv(f"./data/Cleaned/{project_name}.csv", index=False)

9781it [06:28, 25.18it/s]:00<?, ?it/s]
12755it [24:45,  8.59it/s]28<5:11:01, 388.78s/it]
2659it [01:51, 23.95it/s]:14<13:29:59, 1034.03s/it]
15953it [05:47, 45.89it/s]05<7:49:39, 612.60s/it]  
3340it [08:45,  6.36it/s]:53<6:21:06, 508.14s/it]
1145it [00:38, 29.38it/s]:39<6:17:15, 514.44s/it]
2948it [02:37, 18.74it/s]:18<4:12:50, 352.79s/it]
1146it [00:55, 20.61it/s]:55<3:22:14, 288.91s/it]
36514it [02:11, 277.92it/s]1<2:26:40, 214.65s/it]
22124it [57:15,  6.44it/s]03<2:05:56, 188.92s/it]
62190it [16:05, 64.39it/s]51:19<12:54:27, 1191.47s/it]
2390it [01:02, 38.05it/s]:07:27<11:51:07, 1122.84s/it]
3759it [00:33, 112.18it/s]08:30<8:13:34, 800.40s/it]  
38081it [08:53, 71.34it/s]09:03<5:40:51, 568.11s/it]
20757it [04:25, 78.09it/s]17:58<5:25:27, 557.92s/it]
70081it [03:56, 296.23it/s]2:24<4:26:19, 469.99s/it]
35265it [12:54, 45.53it/s] 6:22<3:40:04, 400.14s/it]
36274it [10:38, 56.82it/s]39:17<4:33:35, 512.98s/it]
1958it [00:03, 637.27it/s]49:56<4:44:39, 550.96s/it]
24800it [00:42, 587.70it